In [2]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program, save_languages
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

In [3]:
# CHANGE ME!
TASK_ID = 'row_per_child'
AUTHOR = 'will'

In [5]:
task = Task(
    id=TASK_ID,
    category="Joins",
    name="Row per family to row per child",
    description="For each family, return a row for each child containing the family ID, child ID, DOB, and height.",
    plan=[
        {"id": "each-family", "description": "For each family"},
        {"id": "each-child", "description": "for each child"},
        {"id": "family-ID", "description": "family ID"},
        {"id": "child-ID", "description": "child ID"},
        {"id": "dob-height", "description": "DOB, and height"}
    ],
    sample_input={
        "families": [
            {"family": 1, "dob_child1": "1998", "dob_child2": "1999", "dob_child3": "2000", "height_child1": "5_11", "height_child2": "5_10",  "height_child3": "5_9"},
            {"family": 2, "dob_child1": "1997", "dob_child2": "1995", "dob_child3": "1996", "height_child1": "5_3", "height_child2": "5_5", "height_child3": "5_1"}
        ]
    },
    sample_output=[
        {"family": 1, "child": "child1", "dob": "1998", "height": "5_11"},
        {"family": 1, "child": "child2", "dob": "1999", "height": "5_10"},
        {"family": 1, "child": "child3", "dob": "2000", "height": "5_9"},
        {"family": 2, "child": "child1", "dob": "1997", "height": "5_3"},
        {"family": 2, "child": "child2", "dob": "1995", "height": "5_5"},
        {"family": 2, "child": "child3", "dob": "1996", "height": "5_1"},
    ]
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [22]:
q = replace(
    prototype,
    language='q',
    source='''child_rows: {[child] 
  rows: ?[families; (); 0b; `family`dob`height ! 
    (`family; `$("dob_child",child); `$("height_child", child))];
  update child: (count families)#enlist ("child", child) from rows}
      
row_per_child: (child_rows each ("1"; "2"; "3")) ,/;''').load_plan()
q.execute(task, debug=True)
q.save()

/var/folders/6p/3bzglbgn2ts3v16zf_27zq3r0000gn/T/tmp9zcb8aht
families:("***I***"; enlist ",") 0:`:/var/folders/6p/3bzglbgn2ts3v16zf_27zq3r0000gn/T/tmp9zcb8aht/families.csv
child_rows: {[child] 
  rows: ?[families; (); 0b; `family`dob`height ! 
    (`family; `$("dob_child",child); `$("height_child", child))];
  update child: (count families)#enlist ("child", child) from rows}
      
row_per_child: (child_rows each ("1"; "2"; "3")) ,/;

output: row_per_child
output: output
output: (asc cols output) xcols output
save `:output.csv



In [23]:
q.widget(task)

Output()

CodeWidget(program='{"task": "row_per_child", "language": "q", "plan": {}, "source": "child_rows: {[child] \\n…

In [26]:
r = replace(
    prototype,
    language='r',
    source='''row_per_child <- function(families) {
  families %>%
    pivot_longer(
      !family,
      names_to = c(".value", "child"),
      names_sep = "_",
    )
}''').load_plan()
r.execute(task)
r.save()

In [27]:
r.widget(task)

Output()

CodeWidget(program='{"task": "row_per_child", "language": "r", "plan": {}, "source": "row_per_child <- functio…

In [47]:
sql = replace(
    prototype,
    language='sql',
    source='''SELECT 
  family,
  ('child' || child) AS child,
  (CASE child 
    WHEN 1 THEN dob_child1 
    WHEN 2 THEN dob_child2 
    WHEN 3 THEN dob_child3 
   END) AS dob,
  (CASE child 
    WHEN 1 THEN height_child1 
    WHEN 2 THEN height_child2 
    WHEN 3 THEN height_child3 
   END) AS height
FROM 
  families
  CROSS JOIN
  (SELECT 1 as child UNION VALUES (2), (3))''').load_plan()
sql.execute(task)
sql.save()

In [48]:
sql.widget(task)

Output()

CodeWidget(program='{"task": "row_per_child", "language": "sql", "plan": {"child-ID": [{"line": 2, "start": 2,…

In [34]:
datalog = replace(
    prototype,
    language='datalog',
    source='''row_per_child("child1", dob, family, height) :-
  families(dob, _, _, family, height, _, _).
row_per_child("child2", dob, family, height) :-
  families(_, dob, _, family, _, height, _).
row_per_child("child3", dob, family, height) :-
  families(_, _, dob, family, _, _, height).''').load_plan()
datalog.execute(task, debug=True)
datalog.save()

In [35]:
datalog.widget(task)

Output()

CodeWidget(program='{"task": "row_per_child", "language": "datalog", "plan": {}, "source": "row_per_child(\\"c…

In [36]:
pandas = replace(
    prototype,
    language='python-pandas',
    author='will',
    source='''def row_per_child(families):
  df = pd.wide_to_long(
    families, 
    stubnames=['dob', 'height'], 
    sep="_child", 
    i='family', 
    j='child').reset_index()
  df.child = df.child.map(lambda c: f'child{c}')
  return df''').load_plan()
pandas.execute(task)
pandas.save()

In [37]:
pandas.widget(task)

Output()

CodeWidget(program='{"task": "row_per_child", "language": "python-pandas", "plan": {}, "source": "def row_per_…

In [43]:
python_functional = replace(
    prototype,
    language='python-functional',
    source='''def row_per_child(families):
  return [
    {'family': family['family'],
     'child': f'child{i}',
     'dob': family[f'dob_child{i}'],
     'height': family[f'height_child{i}']}
    for family in families    
    for i in [1, 2, 3]
  ]''').load_plan()
python_functional.execute(task)
python_functional.save()

In [44]:
python_functional.widget(task)

Output()

CodeWidget(program='{"task": "row_per_child", "language": "python-functional", "plan": {}, "source": "def row_…

In [45]:
python_imperative = replace(
    prototype,
    language='python-imperative',
    source='''
def row_per_child(families):
  children = []
  for family in families:
    for i in [1, 2, 3]:
      children.append({
        'family': family['family'],
        'child': f'child{i}',
        'dob': family[f'dob_child{i}'],
        'height': family[f'height_child{i}']
      })
  return children
''').load_plan()
python_imperative.execute(task)
python_imperative.save()

In [46]:
python_imperative.widget(task)

Output()

CodeWidget(program='{"task": "row_per_child", "language": "python-imperative", "plan": {}, "source": "\\ndef r…